# Table of Contents
 <p><div class="lev1"><a href="#Import-Python-modules"><span class="toc-item-num">1 - </span>Import <code>Python</code> modules</a></div><div class="lev1"><a href="#Set-CONSTANTS"><span class="toc-item-num">2 - </span>Set <em>CONSTANTS</em></a></div><div class="lev1"><a href="#List-Exogenous-SymPy-Variables"><span class="toc-item-num">3 - </span>List Exogenous <code>SymPy</code> Variables</a></div><div class="lev1"><a href="#Build-Structure-of-Model-for-Valuation-(conditional-upon-survival)"><span class="toc-item-num">4 - </span>Build Structure of Model for Valuation <em>(conditional upon survival)</em></a></div><div class="lev1"><a href="#Calculate-Actual-Values"><span class="toc-item-num">5 - </span>Calculate Actual Values</a></div>

# Import `Python` modules

In [1]:
from __future__ import print_function
import numpy
import pandas
from pprint import pprint
import sympy
    
from FinSymPy.Valuation import \
    terminal_value as tv, \
    present_value as pv, \
    net_present_value as npv
    
from HelpyFuncs.SymPy import sympy_eval_by_theano

Using gpu device 0: GeForce GT 750M


# Set _CONSTANTS_

In [2]:
# U.S. Corporate Tax rate
US_CORP_TAX_RATE = .48

# Risk-Free Rate
RISK_FREE_RATE = .11

# Public Market Return
PUB_MKT_RETURN = .19

# CAPM Betas
PUB_MKT_BETA_FOR_FIN_SECTOR = 1.   # assumed in line with overall equity market
PEVC_BETA_FOR_PRO_FORMA_PERIOD = 2.

# Long-Term Growth Rate
LONG_TERM_GROWTH_RATE = .08

# Year 0
YEAR_0 = 1980

# Number of Pro-Forma Years EXCLUDING Year 0
NB_PRO_FORMA_YEARS = 3
FINAL_PRO_FORMA_YEAR = YEAR_0 + NB_PRO_FORMA_YEARS

# List Exogenous `SymPy` Variables

In [3]:
# Cash Flow components
Rev = sympy.symbols(
    'Rev_%d:%d' % (YEAR_0, FINAL_PRO_FORMA_YEAR + 1))
print('Revenue variables:\n', Rev)

OpEx___neg = sympy.symbols(
    'OpEx___neg_%d:%d' % (YEAR_0, FINAL_PRO_FORMA_YEAR + 1))
print('\nOperating Expenses (negative) variables:\n', OpEx___neg)

extra_exec_comp___neg = sympy.symbols(
    'extra_exec_comp___neg_%d:%d' % (YEAR_0, FINAL_PRO_FORMA_YEAR + 1))
print('\nExtra Executive Compensation (negative) variables:\n', extra_exec_comp___neg)


# Corporate Tax Rate
corp_tax_rate = sympy.Symbol('corp_tax_rate')
print('\nCorporate Tax Rate variable:\n', corp_tax_rate)


# Long-Term Growth Rate
long_term_growth_rate = sympy.Symbol('long_term_growth_rate')
print('\nLong-Term Growth Rate variable:\n', long_term_growth_rate)


# CAPM Paramaters
risk_free_rate = sympy.Symbol('risk_free_rate')
print('\nRisk-Free Rate variable:\n', risk_free_rate)

pub_mkt_return = sympy.Symbol('pub_mkt_return')
print('\nPublic Market Return variable:\n', pub_mkt_return)

stabilized_beta = sympy.Symbol('stabilized_beta')
print('\nStabilized Beta variable:\n', stabilized_beta)

pevc_beta = sympy.Symbol('pevc_beta')
print('\nPrivate Equity / Venture Capital Beta variable:\n', pevc_beta)


# assemble all exogenous variables into a collection
exo_vars = \
    Rev + OpEx___neg + extra_exec_comp___neg + \
    (corp_tax_rate,
     long_term_growth_rate,
     risk_free_rate, pub_mkt_return,
     stabilized_beta, pevc_beta)

Revenue variables:
 (Rev_1980, Rev_1981, Rev_1982, Rev_1983)

Operating Expenses (negative) variables:
 (OpEx___neg_1980, OpEx___neg_1981, OpEx___neg_1982, OpEx___neg_1983)

Extra Executive Compensation (negative) variables:
 (extra_exec_comp___neg_1980, extra_exec_comp___neg_1981, extra_exec_comp___neg_1982, extra_exec_comp___neg_1983)

Corporate Tax Rate variable:
 corp_tax_rate

Long-Term Growth Rate variable:
 long_term_growth_rate

Risk-Free Rate variable:
 risk_free_rate

Public Market Return variable:
 pub_mkt_return

Stabilized Beta variable:
 stabilized_beta

Private Equity / Venture Capital Beta variable:
 pevc_beta


# Build Structure of Model for Valuation _(conditional upon survival)_

EBIT from Revenue, Operating Expenses and extra executive compensation:

In [4]:
EBIT = map(
    lambda x: reduce(lambda u, v: u + v, x), 
    zip(Rev, OpEx___neg, extra_exec_comp___neg))

print('EBIT =')
pprint(EBIT)

EBIT =
[OpEx___neg_1980 + Rev_1980 + extra_exec_comp___neg_1980,
 OpEx___neg_1981 + Rev_1981 + extra_exec_comp___neg_1981,
 OpEx___neg_1982 + Rev_1982 + extra_exec_comp___neg_1982,
 OpEx___neg_1983 + Rev_1983 + extra_exec_comp___neg_1983]


EBIAT after corporate tax:

In [5]:
EBIAT = map(
    lambda x: (1 - corp_tax_rate) * x,
    EBIT)

print('EBIAT =')
pprint(EBIAT)

EBIAT =
[(-corp_tax_rate + 1)*(OpEx___neg_1980 + Rev_1980 + extra_exec_comp___neg_1980),
 (-corp_tax_rate + 1)*(OpEx___neg_1981 + Rev_1981 + extra_exec_comp___neg_1981),
 (-corp_tax_rate + 1)*(OpEx___neg_1982 + Rev_1982 + extra_exec_comp___neg_1982),
 (-corp_tax_rate + 1)*(OpEx___neg_1983 + Rev_1983 + extra_exec_comp___neg_1983)]


As the case provides no details on projected investments / divestments and changes in working capital, we assume that the FCF before Terminal Value equals the estimated EBIAT:

In [6]:
FCF_before_TV = EBIAT

print('FCF_before_TV =')
pprint(FCF_before_TV)

FCF_before_TV =
[(-corp_tax_rate + 1)*(OpEx___neg_1980 + Rev_1980 + extra_exec_comp___neg_1980),
 (-corp_tax_rate + 1)*(OpEx___neg_1981 + Rev_1981 + extra_exec_comp___neg_1981),
 (-corp_tax_rate + 1)*(OpEx___neg_1982 + Rev_1982 + extra_exec_comp___neg_1982),
 (-corp_tax_rate + 1)*(OpEx___neg_1983 + Rev_1983 + extra_exec_comp___neg_1983)]


Discount Rates:

In [7]:
pro_forma_period_discount_rate = \
    risk_free_rate + pevc_beta * (pub_mkt_return - risk_free_rate)

print(
    'Pro-Forma Period Discount Rate =\n',
    pro_forma_period_discount_rate)

Pro-Forma Period Discount Rate =
 pevc_beta*(pub_mkt_return - risk_free_rate) + risk_free_rate


In [8]:
long_term_discount_rate = \
    risk_free_rate + stabilized_beta * (pub_mkt_return - risk_free_rate)
    
print(
    'Long-Term Discount Rate =\n',
    long_term_discount_rate)

Long-Term Discount Rate =
 risk_free_rate + stabilized_beta*(pub_mkt_return - risk_free_rate)


Terminal Value:

In [9]:
TerminalValue = tv(FCF_before_TV[-1], long_term_discount_rate, long_term_growth_rate)
    
print('TerminalValue =\n', TerminalValue)

TerminalValue =
 (-corp_tax_rate + 1)*(long_term_growth_rate + 1)*(OpEx___neg_1983 + Rev_1983 + extra_exec_comp___neg_1983)/(-long_term_growth_rate + risk_free_rate + stabilized_beta*(pub_mkt_return - risk_free_rate))


Valuation from applying relevant discount rates to _pro forma_ period CFs and to Terminal Value:

In [10]:
val_of_FCF_before_TV = npv(FCF_before_TV, pro_forma_period_discount_rate)

val_of_terminal_val = pv(TerminalValue, long_term_discount_rate, NB_PRO_FORMA_YEARS)

val = val_of_FCF_before_TV + val_of_terminal_val

print('Valuation =\n', val)

Valuation =
 (-corp_tax_rate + 1)*(long_term_growth_rate + 1)*(OpEx___neg_1983 + Rev_1983 + extra_exec_comp___neg_1983)/((-long_term_growth_rate + risk_free_rate + stabilized_beta*(pub_mkt_return - risk_free_rate))*(risk_free_rate + stabilized_beta*(pub_mkt_return - risk_free_rate) + 1)**3) + (-corp_tax_rate + 1)*(OpEx___neg_1980 + Rev_1980 + extra_exec_comp___neg_1980) + (-corp_tax_rate + 1)*(OpEx___neg_1981 + Rev_1981 + extra_exec_comp___neg_1981)/(pevc_beta*(pub_mkt_return - risk_free_rate) + risk_free_rate + 1) + (-corp_tax_rate + 1)*(OpEx___neg_1982 + Rev_1982 + extra_exec_comp___neg_1982)/(pevc_beta*(pub_mkt_return - risk_free_rate) + risk_free_rate + 1)**2 + (-corp_tax_rate + 1)*(OpEx___neg_1983 + Rev_1983 + extra_exec_comp___neg_1983)/(pevc_beta*(pub_mkt_return - risk_free_rate) + risk_free_rate + 1)**3


# Calculate Actual Values

Import data:

In [11]:
data_df = \
    pandas.read_csv(
        'data/Technical Data.csv',
        index_col='Year')

data_df

,Revenue,OpEx,extra_executive_compensation
Year,,,
1981,288000,-242774,-2500
1982,799350,-308991,-75000
1983,1181700,-410010,-140000


Exogenous Variables' values:

In [12]:
exo_vars_values_dict = \
    dict(
        corp_tax_rate=US_CORP_TAX_RATE,
        risk_free_rate=RISK_FREE_RATE,
        pub_mkt_return=PUB_MKT_RETURN,
        pevc_beta=PEVC_BETA_FOR_PRO_FORMA_PERIOD,
        stabilized_beta=PUB_MKT_BETA_FOR_FIN_SECTOR,
        long_term_growth_rate=LONG_TERM_GROWTH_RATE)
    
exo_vars_values_dict.update(
    dict(
        zip([i.name for i in Rev],
            [0] + data_df.Revenue.tolist()) +
        zip([i.name for i in OpEx___neg],
            [0] + data_df.OpEx.tolist()) +
        zip([i.name for i in extra_exec_comp___neg],
            [0] + data_df.extra_executive_compensation.tolist())))

pprint(exo_vars_values_dict)

{'OpEx___neg_1980': 0,
 'OpEx___neg_1981': -242774,
 'OpEx___neg_1982': -308991,
 'OpEx___neg_1983': -410010,
 'Rev_1980': 0,
 'Rev_1981': 288000,
 'Rev_1982': 799350,
 'Rev_1983': 1181700,
 'corp_tax_rate': 0.48,
 'extra_exec_comp___neg_1980': 0,
 'extra_exec_comp___neg_1981': -2500,
 'extra_exec_comp___neg_1982': -75000,
 'extra_exec_comp___neg_1983': -140000,
 'long_term_growth_rate': 0.08,
 'pevc_beta': 2.0,
 'pub_mkt_return': 0.19,
 'risk_free_rate': 0.11,
 'stabilized_beta': 1.0}


Function to calculate values from `SymPy` expression:

In [13]:
def calc(x):
    if isinstance(x, (list, tuple)):
        return [float(calc(i)) for i in x]
    else:
        return float(sympy_eval_by_theano(x, symbols=exo_vars, **exo_vars_values_dict))

Valuation Calculations:

In [14]:
print('VALUATION CONDITIONAL UPON SURVIVAL')
print('___________________________________\n')

print('U.S. Corporate Tax Rate =   %s\n' % '{:.0%}'.format(calc(corp_tax_rate)))

print('Risk-Free Rate =   %s' % '{:.0%}'.format(calc(risk_free_rate)))
print('Public-Market Return =   %s\n' % '{:.0%}'.format(calc(pub_mkt_return)))

print('Stabilized Beta =   %.0fx' % calc(stabilized_beta))
print('Long-Term Discount Rate =   %s\n' % '{:.0%}'.format(calc(long_term_discount_rate)))

print('PEVC Beta =   %.0fx' % calc(pevc_beta))
print('Discount Rate for Pro-Forma Period =   %s\n' % '{:.0%}'.format(calc(pro_forma_period_discount_rate)))

print('Valuation of FCF before TV =   $%.2f MM' % (calc(val_of_FCF_before_TV) / 1e6))
print('Valuation of TV =   $%.2f MM' % (calc(val_of_terminal_val) / 1e6))
print('\nVALUATION | survival =   $%.2f MM\n' % (calc(val) / 1e6))

print('Detailed Cash Flow Calcs:')
val_calcs_df = pandas.DataFrame(index=['Year 0'] + range(YEAR_0 + 1, FINAL_PRO_FORMA_YEAR + 1))
# ref: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html
pandas.options.display.float_format = '{:,.0f}'.format

val_calcs_df['Revenue'] = calc(Rev)
val_calcs_df['Operating Expenses'] = calc(OpEx___neg)
val_calcs_df['Adj. for Extra Exec. Comp.'] = calc(extra_exec_comp___neg)
val_calcs_df['EBIT'] = calc(EBIT)
val_calcs_df['EBIAT'] = calc(EBIAT)
val_calcs_df['FCF before Terminal Value'] = calc(FCF_before_TV)
val_calcs_df['Terminal Value'] = numpy.nan
val_calcs_df.loc[FINAL_PRO_FORMA_YEAR, 'Terminal Value'] = calc(TerminalValue)

val_calcs_df.T

VALUATION CONDITIONAL UPON SURVIVAL
___________________________________

U.S. Corporate Tax Rate =   48%

Risk-Free Rate =   11%
Public-Market Return =   19%

Stabilized Beta =   1x
Long-Term Discount Rate =   19%

PEVC Beta =   2x
Discount Rate for Pro-Forma Period =   27%

Valuation of FCF before TV =   $0.31 MM
Valuation of TV =   $1.91 MM

VALUATION | survival =   $2.23 MM

Detailed Cash Flow Calcs:


,Year 0,1981,1982,1983
Revenue,0,"288,000","799,350","1,181,700"
Operating Expenses,0,"-242,774","-308,991","-410,010"
Adj. for Extra Exec. Comp.,0,"-2,500","-75,000","-140,000"
EBIT,0,"42,726","415,359","631,690"
EBIAT,0,"22,218","215,987","328,479"
FCF before Terminal Value,0,"22,218","215,987","328,479"
Terminal Value,nan,nan,nan,"3,225,064"
